# Suche im Linkkontext
Um neue archivierungswürdige Webinhalte für eine Sammlung zu finden, werten wir die Links in archivierten Websites aus. Mit Hilfe des [Archives Unleashed Toolkit](https://archivesunleashed.org/aut/) haben wir neben dem Linkziel auch den Textkontext des Links aus den archivierten HTML-Dokumenten extrahiert. Zusätzlich haben wir die Links mit thematischen Schlagworten aus dem [OPAC der Bayerischen Staatsbibliothek](https://opacplus.bsb-muenchen.de) angereichert, mit denen Bibliothekar\*innen die archivierten Ausgangswebsites beschrieben haben.  
Nach der Indexierung der Daten in [Apache Solr](https://solr.apache.org/) können Bibliothekar\*innen gezielt im Linkkontext nach Webinhalten zu bestimmten Themen suchen oder anhand des Schlagwortfilters die Links durchstöbern, die von einer archivierten Website zu einem Thema ausgehen.  
Mit den Booleschen Operatoren `AND`, `OR`, `NOT` können Suchbegriffe verknüpft oder ausgeschlossen werden und Suchanfragen so verfeinert werden.

In [ ]:
import requests
import json
from IPython.core.display import display
from ipywidgets import Accordion, HTML, Text, Layout, SelectMultiple, Button
from util.solr_helper.queries import get_facets, get_links
from util.solr_helper.result_formatter import write_domain_table

display(HTML('''<style>
    .widget-label { min-width: 25ex !important; }
</style>
'''))

solr_url = "http://localhost:8988/solr/link-index"
annotations = get_facets(solr_url, "src_annotations")

facets = SelectMultiple(
    options=annotations,
    description='<b>Filter nach Schlagwort: </b>',
    disabled=False,
    layout=Layout(width="70%", height="100%")
)  
default_query = "Dialekt Fränkisch"
query_input = Text(value=default_query, description="<b>Suche nach: </b>", layout=Layout(width="70%"))
acc = Accordion(selected_index=None)

def display_results(b):
    results = get_links(solr_url, query_input.value, facets.value, rows=100)
    domains = results.grouped["domain"]["groups"]
    domain_tables = []
    for i, domain in enumerate(domains):
        acc.set_title(i, f'{i+1}. {domain["groupValue"]}')
        domain_table = write_domain_table(domain, results.highlighting)
        domain_tables.append(HTML(domain_table))
    acc.children = domain_tables

button = Button(description="Suche starten")
button.on_click(display_results)

display(facets, query_input, button, acc)